## Config
---

#### **Imports**

In [23]:
import requests
from datetime import date
import pandas as pd
import csv
import os
import re
from unicodedata import normalize
from decouple import Config, RepositoryEnv
import sqlalchemy
from sqlalchemy import exc

#### **Getting the urls from the config file**

In [18]:
DOTENV_FILE = '.env'
config = Config(RepositoryEnv(DOTENV_FILE))

In [19]:
url_museos = config('URL_MUSEOS')
url_cines = config('URL_CINES')
url_bibliotecas = config('URL_BIBLIOTECAS')
url_database = config('DATABASE_URL')

#### **Connect to the database**

In [20]:
def connect():
    try:
        engine = sqlalchemy.create_engine(config('DATABASE_URL'))
        engine.connect()
        return engine
    except exc.SQLAlchemyError as e:
        return None

## Source files
---

#### **Function to create the name of the csv files with the date**
- Example: categoría\año-mes\categoria-dia-mes-año.csv

In [21]:
def create_path(categoria):
    try:
        name =  categoria + '-' + date.today().strftime('%d-%m-%Y') + '.csv'
        path = categoria+'/'+ date.today().strftime('%Y-%m') 
        if not os.path.exists(path):
            os.makedirs(path)
        return path +'/'+ name
    except Exception as e:
        logging.error('Error al crear el path')
        return None

#### **Creating the names of the csv files**
- Example: museos\2021-noviembre\museos-03-11-2021.csv

In [24]:
name_museos = create_path('museos')
name_cines = create_path('cines')
name_bibliotecas = create_path('bibliotecas')

#### **Header without accent and to lower case**
- Example: Ave María -> ave maria

In [25]:
def normalize_string(s):
    s = re.sub(r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", normalize('NFD', s), 0, re.I)
    return normalize('NFC', s).lower()

In [26]:
def change_header_csv(name):
    try:
        with open(name, 'r') as f:
            reader = csv.reader(f)
            header = next(reader)  
            header = list(map(normalize_string, header))
            rows = [header] + list(reader)  
        with open(name, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerows(rows) 
    except:
        print('Error al cambiar el header')

#### **Downloading the CSV**

In [27]:
def get_csv(url, name):
    try:
        r = requests.get(url)
        with open(name, 'wb') as f:
            f.write(r.content)
        change_header_csv(name)
    except:
        print('Error al descargar el csv')

In [30]:
def download_data():
    get_csv(url_museos, name_museos)
    get_csv(url_cines, name_cines)
    get_csv(url_bibliotecas, name_bibliotecas)

In [29]:
download_data()

## Processing Data
---

#### **Read CSV To dataframe**

In [31]:
df_museos = pd.read_csv(name_museos)
df_cines = pd.read_csv(name_cines)
df_bibliotecas = pd.read_csv(name_bibliotecas)

#### **Creating the DataFrame 1**

In [32]:
def changeHeader(df):
    df.rename({'cod_loc': 'cod_localidad', 'idprovincia': 'id_provincia', 'iddepartamento': 'id_departamento', 'cp': 'cod_postal', 'telefono': 'numero_telefono'}, axis=1, inplace=True)
    return df

In [33]:
def normalize_table(df_museos,df_cines,df_bibliotecas):
    headersBiliotecas = ['cod_loc', 'idprovincia', 'iddepartamento', 'categoria', 'provincia', 'localidad', 'nombre', 'cp', 'telefono', 'mail', 'web','domicilio']
    headers = ['cod_loc', 'idprovincia', 'iddepartamento', 'categoria', 'provincia', 'localidad', 'nombre','cp', 'telefono', 'mail', 'web']
    
    new_df_museos = df_museos[headers]
    new_df_cines = df_cines[headers]
    new_df_bibliotecas = df_bibliotecas[headersBiliotecas]

    #Merge the dataframes
    df_normalize = pd.concat([new_df_museos, new_df_cines, new_df_bibliotecas])
    return changeHeader(df_normalize)

In [34]:
table1 = normalize_table(df_museos,df_cines,df_bibliotecas)

In [35]:
table1

,cod_localidad,id_provincia,id_departamento,categoria,provincia,localidad,nombre,cod_postal,numero_telefono,mail,web,domicilio
0,6588100,6,6588,Espacios de Exhibición Patrimonial,Buenos Aires,9 de Julio,Archivo Y Museo Histórico Gral. Julio De Vedia,B6500EVL,425 279,archivoymuseo@yahoo.com.ar,www.portaldel9.com.ar,NaN
1,6077010,6,6077,Espacios de Exhibición Patrimonial,Buenos Aires,Arrecifes,Museo Y Archivo Histórico De Arrecifes,B2740FMJ,452 931,museoarrecifes@hotmail.com,NaN,NaN
2,6035010,6,6035,Espacios de Exhibición Patrimonial,Buenos Aires,Avellaneda,Museo Histórico-Sacro Hno. Rogelio Scortegagna,S3561AKT,481 200,NaN,NaN,NaN
3,6035010,6,6035,Espacios de Exhibición Patrimonial,Buenos Aires,Avellaneda,Museo Municipal De Artes Plásticas De Avellaneda,B1870CBC,4205 9567,NaN,NaN,NaN
4,6049020,6,6049,Espacios de Exhibición Patrimonial,Buenos Aires,Azul,Museo Etnográfico Y Archivo Histórico Enrique ...,B7300XAA,434 811,museoazul@yahoo.com,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2012,62042560,62,62042,Bibliotecas Populares,Río Negro,Villa Regina,El Progreso,8336,s/d,"helda_33@hotmail.com, bibliotecaelprogreso@hot...",NaN,Pellegrini 476
2013,70028010,70,70028,Bibliotecas Populares,San Juan,"Villa Seminario, Rivadavia",Biblioteca Popular 23 de Septiembre,s/d,4331521,s/d,NaN,Pueyrredón 1692
2014,62007080,62,62007,Bibliotecas Populares,Río Negro,San Javier,Prospero Entraigas,8500,s/d,"biblio_sanjavier@hotmail.com;, andreaaranea03@...",NaN,San Javier
2015,62077010,62,62077,Bibliotecas Populares,Río Negro,Las Grutas,Cmte. Luis Piedra Buena,8521,497263,"bibliotecalasgrutas@yahoo.com.ar, alehernandez...",NaN,Villa Regina 589


#### **Creating the DataFrame 2**

In [36]:
def register_count_table(df_museos,df_cines,df_bibliotecas):
   
    #Get the dataframe with categoria, provincia and fuente
    new_df_bibliotecas = df_bibliotecas[['categoria', 'provincia', 'fuente']]
    new_df_cines2 = df_cines[['categoria', 'provincia', 'fuente']]
    new_df_museos2 = df_museos[['categoria', 'provincia', 'fuente']]

    #Count the registers by categoria, provincia and fuente of bibliotecas
    categoria_biblioteca = new_df_bibliotecas.groupby(['categoria']).size().reset_index(name='registros_categorias')
    fuente_biblioteca = new_df_bibliotecas.groupby(['fuente']).size().reset_index(name='registros_fuentes')
    provincia_categoria_biblioteca = new_df_bibliotecas.groupby(['provincia', 'categoria']).size().reset_index(name='registros_provincia_categoria')

    #Count the registers by categoria, provincia and fuente of cines
    categoria_cine = new_df_cines2.groupby(['categoria']).size().reset_index(name='registros_categorias')
    fuente_cine = new_df_cines2.groupby(['fuente']).size().reset_index(name='registros_fuentes')
    provincia_categoria_cine = new_df_cines2.groupby(['provincia', 'categoria']).size().reset_index(name='registros_provincia_categoria')

    #Count the registers by categoria, provincia and fuente of museos
    categoria_museo = new_df_museos2.groupby(['categoria']).size().reset_index(name='registros_categorias')
    fuente_museo = new_df_museos2.groupby(['fuente']).size().reset_index(name='registros_fuentes')
    provincia_categoria_museo = new_df_museos2.groupby(['provincia', 'categoria']).size().reset_index(name='registros_provincia_categoria')
    
    #Merge or concat the dataframes
    df_normalize = pd.concat([categoria_biblioteca, fuente_biblioteca, provincia_categoria_biblioteca, categoria_cine, fuente_cine, provincia_categoria_cine, categoria_museo, fuente_museo, provincia_categoria_museo])
    return df_normalize
    
    

In [37]:
table2 = register_count_table(df_museos,df_cines,df_bibliotecas)

In [38]:
table2

,categoria,registros_categorias,fuente,registros_fuentes,provincia,registros_provincia_categoria
0,Bibliotecas Populares,2017.0,NaN,NaN,NaN,NaN
0,NaN,NaN,CONABIP,2010.0,NaN,NaN
1,NaN,NaN,Gob. Pcia.,1.0,NaN,NaN
2,NaN,NaN,Secretaria de Cultura Provincial,6.0,NaN,NaN
0,Bibliotecas Populares,NaN,NaN,NaN,Buenos Aires,543.0
...,...,...,...,...,...,...
19,Espacios de Exhibición Patrimonial,NaN,NaN,NaN,Santa Cruz,20.0
20,Espacios de Exhibición Patrimonial,NaN,NaN,NaN,Santa Fe,162.0
21,Espacios de Exhibición Patrimonial,NaN,NaN,NaN,Santiago del Estero,10.0
22,Espacios de Exhibición Patrimonial,NaN,NaN,NaN,"Tierra del Fuego, Antártida e Islas del Atlánt...",14.0


#### **Creating the DataFrame 3**

In [39]:
def info_cine():
   df_cines2 = df_cines[['provincia', 'pantallas', 'butacas', 'espacio_incaa']]
   return df_cines2

In [40]:
table3 = info_cine()

In [41]:
table3

,provincia,pantallas,butacas,espacio_incaa
0,Catamarca,5,743,NaN
1,Catamarca,1,440,0
2,Chaco,5,820,NaN
3,Chubut,1,80,NaN
4,Corrientes,1,240,NaN
...,...,...,...,...
324,Buenos Aires,1,0,NaN
325,Buenos Aires,8,2037,NaN
326,Buenos Aires,1,430,si
327,Catamarca,1,1103,si


## Creation of tables in the Database
---

In [42]:
def upload_to_db(df, name):
    try:
        engine = connect()
        if(engine):
            df.to_sql(name, engine, if_exists='append', index=False)
            print('Se ha subido el dataframe ' + name + ' a la base de datos')
            return df
    except exc.SQLAlchemyError as e:
        print('Error al subir el dataframe ' + name + ' a la base de datos')

In [43]:
upload_to_db(table1,'table1')

Se ha subido el dataframe table1 a la base de datos


,cod_localidad,id_provincia,id_departamento,categoria,provincia,localidad,nombre,cod_postal,numero_telefono,mail,web,domicilio
0,6588100,6,6588,Espacios de Exhibición Patrimonial,Buenos Aires,9 de Julio,Archivo Y Museo Histórico Gral. Julio De Vedia,B6500EVL,425 279,archivoymuseo@yahoo.com.ar,www.portaldel9.com.ar,NaN
1,6077010,6,6077,Espacios de Exhibición Patrimonial,Buenos Aires,Arrecifes,Museo Y Archivo Histórico De Arrecifes,B2740FMJ,452 931,museoarrecifes@hotmail.com,NaN,NaN
2,6035010,6,6035,Espacios de Exhibición Patrimonial,Buenos Aires,Avellaneda,Museo Histórico-Sacro Hno. Rogelio Scortegagna,S3561AKT,481 200,NaN,NaN,NaN
3,6035010,6,6035,Espacios de Exhibición Patrimonial,Buenos Aires,Avellaneda,Museo Municipal De Artes Plásticas De Avellaneda,B1870CBC,4205 9567,NaN,NaN,NaN
4,6049020,6,6049,Espacios de Exhibición Patrimonial,Buenos Aires,Azul,Museo Etnográfico Y Archivo Histórico Enrique ...,B7300XAA,434 811,museoazul@yahoo.com,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2012,62042560,62,62042,Bibliotecas Populares,Río Negro,Villa Regina,El Progreso,8336,s/d,"helda_33@hotmail.com, bibliotecaelprogreso@hot...",NaN,Pellegrini 476
2013,70028010,70,70028,Bibliotecas Populares,San Juan,"Villa Seminario, Rivadavia",Biblioteca Popular 23 de Septiembre,s/d,4331521,s/d,NaN,Pueyrredón 1692
2014,62007080,62,62007,Bibliotecas Populares,Río Negro,San Javier,Prospero Entraigas,8500,s/d,"biblio_sanjavier@hotmail.com;, andreaaranea03@...",NaN,San Javier
2015,62077010,62,62077,Bibliotecas Populares,Río Negro,Las Grutas,Cmte. Luis Piedra Buena,8521,497263,"bibliotecalasgrutas@yahoo.com.ar, alehernandez...",NaN,Villa Regina 589


In [44]:
upload_to_db(table2,'table2')

Se ha subido el dataframe table2 a la base de datos


,categoria,registros_categorias,fuente,registros_fuentes,provincia,registros_provincia_categoria
0,Bibliotecas Populares,2017.0,NaN,NaN,NaN,NaN
0,NaN,NaN,CONABIP,2010.0,NaN,NaN
1,NaN,NaN,Gob. Pcia.,1.0,NaN,NaN
2,NaN,NaN,Secretaria de Cultura Provincial,6.0,NaN,NaN
0,Bibliotecas Populares,NaN,NaN,NaN,Buenos Aires,543.0
...,...,...,...,...,...,...
19,Espacios de Exhibición Patrimonial,NaN,NaN,NaN,Santa Cruz,20.0
20,Espacios de Exhibición Patrimonial,NaN,NaN,NaN,Santa Fe,162.0
21,Espacios de Exhibición Patrimonial,NaN,NaN,NaN,Santiago del Estero,10.0
22,Espacios de Exhibición Patrimonial,NaN,NaN,NaN,"Tierra del Fuego, Antártida e Islas del Atlánt...",14.0


In [45]:
upload_to_db(table3,'table3')

Se ha subido el dataframe table3 a la base de datos


,provincia,pantallas,butacas,espacio_incaa
0,Catamarca,5,743,NaN
1,Catamarca,1,440,0
2,Chaco,5,820,NaN
3,Chubut,1,80,NaN
4,Corrientes,1,240,NaN
...,...,...,...,...
324,Buenos Aires,1,0,NaN
325,Buenos Aires,8,2037,NaN
326,Buenos Aires,1,430,si
327,Catamarca,1,1103,si
